# 🏭 New Machine Commissioning Advisor
## JARVIS — Smart Factory Monitoring System

**AI-Powered Advisory System for New Machine Setup & Configuration**

---

| | |
|---|---|
| **Author** | Rithish |
| **Date** | February 17, 2026 |
| **Version** | 2.0.0 |
| **Tech Stack** | FastAPI, React 19, SQLAlchemy, Python 3.10+ |

---

## 📑 Table of Contents

1. [Abstract](#abstract)
2. [Introduction](#introduction)
3. [Objectives](#objectives)
4. [System Architecture](#architecture)
5. [Data Models](#models)
6. [Backend Service — Core Algorithms](#service)
7. [API Endpoints](#api)
8. [Frontend Implementation](#frontend)
9. [Testing & Results](#testing)
10. [Key Design Decisions](#decisions)
11. [Conclusion & Future Scope](#conclusion)

---
<a name='abstract'></a>
## 1. Abstract

This report presents the implementation of the **New Machine Commissioning Advisor**, a feature module within the JARVIS Smart Factory Monitoring System. The advisor leverages historical factory sensor data and alert records to generate intelligent, data-driven recommendations when commissioning new machines on the factory floor.

The system employs **statistical analysis** on historical sensor readings (temperature, vibration, RPM) and **alert pattern recognition** to produce actionable outputs: safe operating limits, operational risk assessments, configuration recommendations, and preventive maintenance guidelines.

The implementation follows a full-stack architecture with a **FastAPI** backend, **SQLAlchemy** ORM, and a **React 19** frontend with a futuristic HUD-style user interface.

---
<a name='introduction'></a>
## 2. Introduction

Smart manufacturing environments generate vast amounts of sensor data over time. When a new machine is introduced to the factory floor, operators typically rely on manufacturer-provided default configurations and generic guidelines. This approach ignores the wealth of knowledge embedded in the factory's own operational history.

The **New Machine Commissioning Advisor** bridges this gap by analyzing historical data from the factory's existing machines to:

- Identify recurring **failure patterns** and their root causes
- Calculate statistically-derived **safe operating thresholds**
- Assess **risks** specific to the new machine's specifications
- Recommend optimal **monitoring configurations**
- Generate **preventive maintenance guidelines**

> **Key Innovation:** Unlike static commissioning checklists, this system provides *dynamic, factory-specific* recommendations that evolve as more operational data is accumulated.

---
<a name='objectives'></a>
## 3. Objectives

1. **Historical Pattern Mining:** Analyze past alerts to identify common failure types, severities, and frequencies across all machines.
2. **Statistical Limit Calculation:** Compute optimal, warning, and critical thresholds for temperature, vibration, and RPM based on real sensor data from healthy machines.
3. **Risk-Aware Commissioning:** Cross-reference incoming machine specifications with historical incident data to flag potential risks.
4. **Actionable Recommendations:** Generate specific configuration parameters and preventive guidelines tailored to the new machine's profile.
5. **Intuitive User Interface:** Present analysis results in a clear, visually organized dashboard.

---
<a name='architecture'></a>
## 4. System Architecture

### 4.1 High-Level Architecture

```
┌─────────────────────┐     ┌──────────────────────┐     ┌──────────────────────┐     ┌─────────────────┐
│  React Frontend     │────▶│  FastAPI Router       │────▶│  Service Layer       │────▶│  SQLite DB      │
│  CommissioningPanel │◀────│  api/commissioning.py │◀────│  services/           │◀────│  MachineReading │
│  .jsx               │     │  3 REST endpoints     │     │  commissioning.py    │     │  Alert          │
└─────────────────────┘     └──────────────────────┘     └──────────────────────┘     └─────────────────┘
```

### 4.2 Component Inventory

| Layer | File | Lines | Purpose |
|-------|------|-------|---------|
| Frontend | `CommissioningPanel.jsx` | 339 | Input form + results display |
| API Router | `api/commissioning.py` | 143 | 3 REST endpoints |
| Service | `services/commissioning.py` | 272 | Core analysis algorithms |
| Models | `models/machine_data.py` | 31 | ORM class definitions |
| App Entry | `main.py` | 64 | Router registration |
| App Shell | `App.jsx` | 1225 | Navigation + panel integration |

---
## 📦 Install Dependencies

In [ ]:
!pip install fastapi sqlalchemy pydantic uvicorn

---
<a name='models'></a>
## 5. Data Models

The system relies on two SQLAlchemy ORM models that store historical factory data.

In [ ]:
# ===== Data Models (models/machine_data.py) =====

from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from datetime import datetime

Base = declarative_base()


class MachineReading(Base):
    """Historical machine sensor readings"""
    __tablename__ = "machine_readings"

    id = Column(Integer, primary_key=True, index=True)
    machine_id = Column(String, index=True, nullable=False)
    temperature = Column(Float, default=0)       # °C
    vibration = Column(Float, default=0)          # mm/s
    rpm = Column(Float, default=0)                # Revolutions per minute
    health_score = Column(Integer, default=100)   # 0-100%
    status = Column(String, default="Healthy")    # Healthy | Warning | Critical
    timestamp = Column(DateTime, default=datetime.utcnow, index=True)


class Alert(Base):
    """Machine alerts and notifications"""
    __tablename__ = "alerts"

    id = Column(Integer, primary_key=True, index=True)
    machine_id = Column(String, index=True, nullable=False)
    alert_type = Column(String, nullable=False)    # temperature, vibration, rpm, health
    severity = Column(String, nullable=False)      # warning, critical
    message = Column(String, nullable=False)
    acknowledged = Column(Boolean, default=False)
    created_at = Column(DateTime, default=datetime.utcnow, index=True)


# Initialize in-memory database for demonstration
engine = create_engine("sqlite:///factory_demo.db")
Base.metadata.create_all(engine)
SessionLocal = sessionmaker(bind=engine)

print("✅ Database models created successfully")
print(f"   Tables: {list(Base.metadata.tables.keys())}")

### 5.1 Seed Sample Factory Data for Demonstration

In [ ]:
# ===== Generate Sample Historical Data =====

import random
from datetime import timedelta

db = SessionLocal()

MACHINES = ['MILL-01', 'LATHE-02', 'PRESS-03', 'CNC-04', 'DRILL-05']

# Generate 500 sensor readings over 30 days
print('Generating 500 sensor readings...')
for i in range(500):
    machine = random.choice(MACHINES)
    health = random.randint(40, 100)
    status = 'Healthy' if health > 70 else ('Warning' if health > 40 else 'Critical')
    reading = MachineReading(
        machine_id=machine,
        temperature=round(random.gauss(55, 12), 2),
        vibration=round(random.gauss(0.4, 0.15), 3),
        rpm=round(random.gauss(1800, 300), 1),
        health_score=health,
        status=status,
        timestamp=datetime.utcnow() - timedelta(days=random.uniform(0, 30))
    )
    db.add(reading)

# Generate 50 alerts
print('Generating 50 alerts...')
alert_types = ['temperature', 'vibration', 'rpm', 'health']
severities = ['warning', 'critical']
messages = {
    'temperature': ['High temperature detected: {v}°C', 'Temperature spike on {m}'],
    'vibration': ['Excessive vibration: {v}mm/s', 'Vibration anomaly on {m}'],
    'rpm': ['RPM fluctuation detected', 'Abnormal RPM on {m}'],
    'health': ['Health score dropped below threshold', 'Degraded health on {m}']
}

for i in range(50):
    atype = random.choice(alert_types)
    machine = random.choice(MACHINES)
    alert = Alert(
        machine_id=machine,
        alert_type=atype,
        severity=random.choice(severities),
        message=random.choice(messages[atype]).format(v=round(random.uniform(70, 95), 1), m=machine),
        created_at=datetime.utcnow() - timedelta(days=random.uniform(0, 30))
    )
    db.add(alert)

db.commit()
print(f'\n✅ Sample data generated:')
print(f'   Readings: {db.query(MachineReading).count()}')
print(f'   Alerts:   {db.query(Alert).count()}')

---
<a name='service'></a>
## 6. Backend Service — Core Algorithms

The service layer contains **4 core functions** that form the analysis pipeline:

| Function | Purpose |
|----------|---------|
| `identify_failure_patterns()` | Mine historical alerts for recurring failure types |
| `calculate_safe_limits()` | Derive statistical operating thresholds (μ ± kσ) |
| `analyze_operational_risks()` | Cross-reference machine specs with incident history |
| `generate_recommendations()` | Orchestrate all analyses into final advisory output |

In [ ]:
# ===== Commissioning Advisor Service (services/commissioning.py) =====

from typing import Dict, List
from statistics import mean, stdev
from sqlalchemy.orm import Session


def get_all_historical_readings(db: Session, days: int = 30):
    """Get all historical readings for analysis"""
    cutoff = datetime.utcnow() - timedelta(days=days)
    return db.query(MachineReading).filter(
        MachineReading.timestamp >= cutoff
    ).order_by(MachineReading.timestamp.desc()).all()


def get_all_alerts(db: Session, days: int = 30):
    """Get all alerts for pattern analysis"""
    cutoff = datetime.utcnow() - timedelta(days=days)
    return db.query(Alert).filter(
        Alert.created_at >= cutoff
    ).order_by(Alert.created_at.desc()).all()


def get_probable_root_cause(alert_type: str) -> str:
    """Get probable root cause for an alert type"""
    causes = {
        "temperature": "Cooling system inefficiency, bearing friction, or excessive load",
        "vibration": "Mechanical imbalance, loose components, worn bearings, or misalignment",
        "rpm": "Motor control issues, power fluctuations, or mechanical resistance",
        "health": "Cumulative wear, deferred maintenance, or environmental factors"
    }
    return causes.get(alert_type, "Multiple contributing factors - requires investigation")


def get_prevention_tip(alert_type: str) -> str:
    """Get prevention tip for an alert type"""
    tips = {
        "temperature": "Ensure adequate ventilation, regular coolant checks, and monitor ambient temperature",
        "vibration": "Schedule regular alignment checks, secure all mounting bolts, and lubricate bearings",
        "rpm": "Verify power supply stability, check belt/gear conditions, and calibrate motor controllers",
        "health": "Follow preventive maintenance schedule and address minor issues promptly"
    }
    return tips.get(alert_type, "Implement regular inspection and monitoring protocols")


print("✅ Helper functions defined")

### 6.1 Function: `identify_failure_patterns()`

Mines historical alerts to find the most common failure types. Groups by `(alert_type, severity)`, counts occurrences, and enriches with root cause analysis.

In [ ]:
def identify_failure_patterns(db: Session, days: int = 30) -> List[Dict]:
    """
    Identify common failure patterns from historical alerts.
    Groups alerts by type and severity to find recurring issues.
    """
    alerts = get_all_alerts(db, days)
    if not alerts:
        return []

    # Group alerts by type
    pattern_counts = {}
    for alert in alerts:
        key = (alert.alert_type, alert.severity)
        if key not in pattern_counts:
            pattern_counts[key] = {
                "type": alert.alert_type,
                "severity": alert.severity,
                "count": 0,
                "machines_affected": set(),
                "sample_messages": []
            }
        pattern_counts[key]["count"] += 1
        pattern_counts[key]["machines_affected"].add(alert.machine_id)
        if len(pattern_counts[key]["sample_messages"]) < 3:
            pattern_counts[key]["sample_messages"].append(alert.message)

    # Convert to list and sort by frequency
    patterns = []
    for key, data in pattern_counts.items():
        patterns.append({
            "type": data["type"],
            "severity": data["severity"],
            "occurrences": data["count"],
            "machines_affected": len(data["machines_affected"]),
            "frequency": "high" if data["count"] > 10 else ("medium" if data["count"] > 5 else "low"),
            "sample_messages": data["sample_messages"],
            "root_cause": get_probable_root_cause(data["type"]),
            "prevention": get_prevention_tip(data["type"])
        })

    patterns.sort(key=lambda x: x["occurrences"], reverse=True)
    return patterns[:10]


# Test it
patterns = identify_failure_patterns(db)
print(f'✅ Found {len(patterns)} failure patterns\n')
for p in patterns[:5]:
    print(f'  [{p["severity"].upper():8s}] {p["type"]:12s} — {p["occurrences"]} occurrences, {p["machines_affected"]} machines')
    print(f'           Root Cause: {p["root_cause"]}')
    print(f'           Prevention: {p["prevention"]}\n')

### 6.2 Function: `calculate_safe_limits()`

Derives operating thresholds using statistical analysis: **Optimal = μ**, **Warning = μ + σ**, **Critical = μ + kσ**. Only uses data from healthy machines (`health_score > 60`).

In [ ]:
def calculate_safe_limits(db: Session, days: int = 30) -> Dict:
    """
    Calculate safe operating limits based on historical data.
    Uses statistical analysis (mean ± k*stdev) to determine optimal thresholds.
    """
    readings = get_all_historical_readings(db, days)

    if len(readings) < 10:
        return {
            "temperature": {"min": 20, "optimal": 45, "warning": 70, "critical": 85},
            "vibration": {"min": 0.05, "optimal": 0.3, "warning": 0.6, "critical": 0.9},
            "rpm": {"min": 500, "optimal": 1800, "warning": 2800, "critical": 3200},
            "data_points": len(readings), "confidence": "low"
        }

    # Filter only healthy machine data
    temps = [r.temperature for r in readings if r.temperature and r.health_score > 60]
    vibs = [r.vibration for r in readings if r.vibration and r.health_score > 60]
    rpms = [r.rpm for r in readings if r.rpm and r.health_score > 60]

    def calc_limits(values, critical_multiplier=1.5):
        if not values or len(values) < 5:
            return None
        avg = mean(values)
        std = stdev(values) if len(values) > 1 else avg * 0.1
        return {
            "min": round(max(0, avg - 2 * std), 2),
            "optimal": round(avg, 2),
            "warning": round(avg + std, 2),
            "critical": round(avg + critical_multiplier * std, 2)
        }

    limits = {
        "temperature": calc_limits(temps, 2.0) or {"min": 20, "optimal": 45, "warning": 70, "critical": 85},
        "vibration": calc_limits(vibs, 2.5) or {"min": 0.05, "optimal": 0.3, "warning": 0.6, "critical": 0.9},
        "rpm": calc_limits(rpms, 1.5) or {"min": 500, "optimal": 1800, "warning": 2800, "critical": 3200},
        "data_points": len(readings),
        "confidence": "high" if len(readings) > 1000 else ("medium" if len(readings) > 100 else "low")
    }
    return limits


# Test it
limits = calculate_safe_limits(db)
print(f'✅ Safe Operating Limits (from {limits["data_points"]} readings, confidence: {limits["confidence"]})\n')
for param in ['temperature', 'vibration', 'rpm']:
    l = limits[param]
    units = {'temperature': '°C', 'vibration': 'mm/s', 'rpm': 'RPM'}
    print(f'  {param.upper():12s}  Min: {l["min"]:>8} | Optimal: {l["optimal"]:>8} | Warning: {l["warning"]:>8} | Critical: {l["critical"]:>8} {units[param]}')

### 6.3 Function: `analyze_operational_risks()`

Cross-references the new machine's specs with historical incident data to identify potential risks.

In [ ]:
def analyze_operational_risks(db: Session, machine_specs: Dict) -> List[Dict]:
    """Analyze operational risks based on new machine specs and factory history"""
    risks = []
    alerts = get_all_alerts(db, days=30)

    # Power-related risks
    rated_power = machine_specs.get("rated_power", 0)
    if rated_power > 20:
        risks.append({
            "category": "Power",
            "level": "medium",
            "description": f"High power machine ({rated_power}kW) may stress electrical infrastructure",
            "mitigation": "Verify power supply capacity and consider dedicated circuit"
        })

    # RPM-related risks
    rated_rpm = machine_specs.get("rated_rpm", 0)
    rpm_alerts = len([a for a in alerts if a.alert_type == "rpm"])
    if rpm_alerts > 5 and rated_rpm > 2500:
        risks.append({
            "category": "RPM",
            "level": "high",
            "description": f"Factory has history of RPM issues. New machine rated at {rated_rpm} RPM",
            "mitigation": "Implement vibration monitoring and gradual spinup procedures"
        })

    # Environment compatibility
    environment = machine_specs.get("environment", "normal")
    temp_alerts = len([a for a in alerts if a.alert_type == "temperature"])
    if environment == "high_humidity" and temp_alerts > 3:
        risks.append({
            "category": "Environment",
            "level": "high",
            "description": "High humidity + historical temperature issues = condensation risk",
            "mitigation": "Install dehumidifier and monitor for moisture buildup"
        })
    elif environment == "dusty":
        risks.append({
            "category": "Environment",
            "level": "medium",
            "description": "Dusty environment increases filter clogging and overheating risk",
            "mitigation": "Install additional air filtration and schedule frequent cleaning"
        })

    # Vibration risks
    vib_alerts = len([a for a in alerts if a.alert_type == "vibration"])
    if vib_alerts > 5:
        risks.append({
            "category": "Vibration",
            "level": "medium",
            "description": f"Factory has {vib_alerts} vibration alerts. New machine may be affected",
            "mitigation": "Ensure proper foundation and anti-vibration mounting pads"
        })

    # Sort by risk level
    level_order = {"high": 0, "medium": 1, "low": 2}
    risks.sort(key=lambda x: level_order.get(x["level"], 3))
    return risks


# Test it
test_specs = {'machine_type': 'CNC Lathe', 'rated_power': 15, 'rated_rpm': 3000, 'environment': 'high_humidity'}
risks = analyze_operational_risks(db, test_specs)
print(f'✅ Found {len(risks)} operational risks for CNC Lathe\n')
for r in risks:
    emoji = '🔴' if r['level'] == 'high' else '🟡' if r['level'] == 'medium' else '🟢'
    print(f'  {emoji} [{r["level"].upper():6s}] {r["category"]}: {r["description"]}')
    print(f'           Mitigation: {r["mitigation"]}\n')

### 6.4 Function: `generate_recommendations()`

Orchestrates all analyses and produces the final comprehensive advisory output with configuration recommendations and preventive guidelines.

In [ ]:
def generate_recommendations(db: Session, machine_specs: Dict) -> Dict:
    """Generate comprehensive recommendations for new machine commissioning"""
    patterns = identify_failure_patterns(db)
    safe_limits = calculate_safe_limits(db)
    risks = analyze_operational_risks(db, machine_specs)

    # Generate configuration recommendations
    config_recommendations = []

    # Temperature configuration
    temp_limits = safe_limits.get("temperature", {})
    config_recommendations.append({
        "parameter": "Temperature Alert Threshold",
        "recommended_value": temp_limits.get("warning", 70),
        "unit": "°C",
        "reason": f"Based on factory average of {temp_limits.get('optimal', 45)}°C"
    })

    # Vibration configuration
    vib_limits = safe_limits.get("vibration", {})
    config_recommendations.append({
        "parameter": "Vibration Alert Threshold",
        "recommended_value": vib_limits.get("warning", 0.6),
        "unit": "mm/s",
        "reason": f"Factory optimal vibration is {vib_limits.get('optimal', 0.3)} mm/s"
    })

    # RPM configuration
    rated_rpm = machine_specs.get("rated_rpm", 2000)
    config_recommendations.append({
        "parameter": "Max Operating RPM",
        "recommended_value": min(rated_rpm * 0.9, safe_limits["rpm"]["warning"]),
        "unit": "RPM",
        "reason": "10% safety margin from rated capacity"
    })

    # Generate preventive guidelines
    preventive_guidelines = [
        "Perform initial calibration and alignment verification before production",
        "Run break-in period at 50% load for first 24 hours",
        "Monitor all sensors continuously during first week",
        "Schedule preventive maintenance based on factory historical failure patterns"
    ]

    for pattern in patterns[:3]:
        if pattern["severity"] == "critical":
            preventive_guidelines.append(
                f"PRIORITY: Address {pattern['type']} monitoring - {pattern['occurrences']} historical incidents"
            )

    result = {
        "machine_type": machine_specs.get("machine_type", "Unknown"),
        "analysis_timestamp": datetime.utcnow().isoformat(),
        "failure_patterns": patterns,
        "safe_operating_limits": safe_limits,
        "operational_risks": risks,
        "configuration_recommendations": config_recommendations,
        "preventive_guidelines": preventive_guidelines,
        "summary": {
            "total_patterns_found": len(patterns),
            "risk_count": {
                "high": len([r for r in risks if r["level"] == "high"]),
                "medium": len([r for r in risks if r["level"] == "medium"]),
                "low": len([r for r in risks if r["level"] == "low"])
            },
            "data_confidence": safe_limits.get("confidence", "low")
        }
    }
    return result


print("✅ generate_recommendations() defined")

---
<a name='api'></a>
## 7. API Endpoints

The FastAPI router exposes 3 REST endpoints:

| Method | Endpoint | Description |
|--------|----------|-------------|
| `POST` | `/commissioning/analyze` | Full analysis with machine specs |
| `GET` | `/commissioning/patterns` | Historical failure patterns |
| `GET` | `/commissioning/limits` | Safe operating limits |
| `POST` | `/commissioning/risks` | Risk assessment only |

In [ ]:
# ===== API Router (api/commissioning.py) =====

from pydantic import BaseModel
from typing import Optional


class NewMachineSpecs(BaseModel):
    """Specifications for the new machine being commissioned"""
    machine_type: str = "General"
    rated_power: Optional[float] = 10.0       # kW
    rated_rpm: Optional[float] = 2000.0
    rated_temperature: Optional[float] = 60.0  # Max operating temp °C
    environment: Optional[str] = "normal"      # normal, high_humidity, dusty, hot, cold


class CommissioningResponse(BaseModel):
    """Response from commissioning analysis"""
    success: bool
    machine_type: str
    failure_patterns: list
    safe_operating_limits: dict
    operational_risks: list
    configuration_recommendations: list
    preventive_guidelines: list
    summary: dict


# In production, this is a FastAPI route:
# @router.post('/commissioning/analyze', response_model=CommissioningResponse)
# def analyze_for_new_machine(specs: NewMachineSpecs, db: Session = Depends(get_db)):
#     result = generate_recommendations(db, specs.dict())
#     return CommissioningResponse(success=True, **result)

print("✅ API models defined")
print(f"   Request schema:  {list(NewMachineSpecs.model_fields.keys())}")
print(f"   Response schema: {list(CommissioningResponse.model_fields.keys())}")

---
<a name='testing'></a>
## 8. Testing & Results

### 8.1 Full End-to-End Analysis Test

In [ ]:
# ===== Full End-to-End Test =====

test_machine = {
    "machine_type": "CNC Lathe",
    "rated_power": 15,
    "rated_rpm": 3000,
    "rated_temperature": 70,
    "environment": "high_humidity"
}

print('='*70)
print('  🏭 NEW MACHINE COMMISSIONING ADVISOR — FULL ANALYSIS')
print('='*70)
print(f'\n📋 Machine:  {test_machine["machine_type"]}')
print(f'   Power:    {test_machine["rated_power"]} kW')
print(f'   RPM:      {test_machine["rated_rpm"]}')
print(f'   Max Temp: {test_machine["rated_temperature"]}°C')
print(f'   Environ:  {test_machine["environment"]}')

# Run full analysis
result = generate_recommendations(db, test_machine)

print(f'\n{"="*70}')
print(f'  📊 ANALYSIS SUMMARY')
print(f'{"="*70}')
s = result['summary']
print(f'  Patterns Found:  {s["total_patterns_found"]}')
print(f'  High Risks:      {s["risk_count"]["high"]}')
print(f'  Medium Risks:    {s["risk_count"]["medium"]}')
print(f'  Data Confidence: {s["data_confidence"]}')

print(f'\n{"-"*70}')
print('  ⚠️  OPERATIONAL RISKS')
print(f'{"-"*70}')
for r in result['operational_risks']:
    emoji = '🔴' if r['level'] == 'high' else '🟡'
    print(f'  {emoji} [{r["level"].upper()}] {r["category"]}: {r["description"]}')
    print(f'     💡 {r["mitigation"]}')

print(f'\n{"-"*70}')
print('  🛡️  SAFE OPERATING LIMITS')
print(f'{"-"*70}')
units = {'temperature': '°C', 'vibration': 'mm/s', 'rpm': 'RPM'}
for param in ['temperature', 'vibration', 'rpm']:
    l = result['safe_operating_limits'][param]
    print(f'  {param.upper():12s}  Optimal: {l["optimal"]:>8} | Warning: {l["warning"]:>8} | Critical: {l["critical"]:>8} {units[param]}')

print(f'\n{"-"*70}')
print('  ⚙️  CONFIGURATION RECOMMENDATIONS')
print(f'{"-"*70}')
for rec in result['configuration_recommendations']:
    print(f'  • {rec["parameter"]}: {rec["recommended_value"]} {rec["unit"]}')
    print(f'    Reason: {rec["reason"]}')

print(f'\n{"-"*70}')
print('  ✅ PREVENTIVE GUIDELINES')
print(f'{"-"*70}')
for i, g in enumerate(result['preventive_guidelines'], 1):
    print(f'  {i}. {g}')

print(f'\n{"-"*70}')
print('  📋 TOP FAILURE PATTERNS')
print(f'{"-"*70}')
for p in result['failure_patterns'][:5]:
    print(f'  [{p["severity"].upper():8s}] {p["type"]:12s} — {p["occurrences"]} occurrences')
    print(f'           Root Cause: {p["root_cause"]}')

print(f'\n{"="*70}')
print('  ✅ ANALYSIS COMPLETE')
print(f'{"="*70}')

---
<a name='frontend'></a>
## 9. Frontend Implementation

The `CommissioningPanel.jsx` React component provides the user interface:

### Input Form (Left Column)
| Field | Type | Default |
|-------|------|--------|
| Machine Type | Text | (required) |
| Rated Power | Number | 10 kW |
| Rated RPM | Number | 2000 |
| Max Operating Temp | Number | 60°C |
| Environment | Dropdown | normal |

### Results Display (Right Columns)
1. **Analysis Summary** — 4 metric cards
2. **Operational Risks** — Color-coded risk cards with mitigations
3. **Safe Operating Limits** — Temperature, vibration, RPM thresholds
4. **Configuration Recommendations** — Suggested parameters
5. **Historical Failure Patterns** — Top recurring issues

### Navigation
Accessed via the **🏭 NEW** button in the main dashboard header.

In [ ]:
# ===== Frontend Code: CommissioningPanel.jsx (Key Sections) =====

frontend_code = """
import { useState } from "react";
import { Activity, AlertTriangle, CheckCircle, Settings, Shield, Zap } from "lucide-react";

const API_BASE = 'http://127.0.0.1:8000';

export default function CommissioningPanel({ onBack }) {
  const [specs, setSpecs] = useState({
    machine_type: "",
    rated_power: 10,
    rated_rpm: 2000,
    rated_temperature: 60,
    environment: "normal"
  });
  const [analysis, setAnalysis] = useState(null);
  const [loading, setLoading] = useState(false);

  const handleAnalyze = async () => {
    setLoading(true);
    const res = await fetch(`${API_BASE}/commissioning/analyze`, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(specs)
    });
    const data = await res.json();
    setAnalysis(data);
    setLoading(false);
  };

  // ... renders input form + 5 result sections
}
"""

print("📄 Frontend Component: CommissioningPanel.jsx")
print(f"   Total Lines: 339")
print(f"   Features: Input form, API integration, 5 result sections")
print(f"   Navigation: '🏭 NEW' button in App.jsx header")
print(frontend_code)

---
<a name='decisions'></a>
## 10. Key Design Decisions

| Decision | Rationale |
|----------|----------|
| Use **μ ± kσ** for thresholds | Data-driven limits that adapt to factory's own data distribution |
| Filter readings where **health_score > 60** | Learn only from healthy operating conditions |
| **Default fallback** values when data < 10 | Prevents failure on fresh installations |
| Cap patterns at **top 10** | Focuses attention on most impactful issues |
| **90% RPM safety margin** | Standard engineering practice — 10% below rated capacity |
| **Environment-aware** risk analysis | Combines machine specs with historical incidents to find compound risks |
| **Full-screen overlay** rendering | Dedicated workspace without disrupting dashboard |

---
<a name='conclusion'></a>
## 11. Conclusion & Future Scope

### Conclusion
The New Machine Commissioning Advisor has been successfully implemented as a full-stack feature within JARVIS. Key achievements:

- **Statistical analysis pipeline** that adapts thresholds to the factory's own operational profile
- **Multi-factor risk assessment** combining machine specifications with historical incident data
- **Clean, intuitive UI** integrated seamlessly into the existing JARVIS dashboard
- **RESTful API** supporting both integrated panel use and standalone API access

### Future Enhancements
| Enhancement | Description |
|-------------|-------------|
| LLM-Powered Analysis | Integrate Gemini/Ollama for natural language recommendations |
| ML Models | Train regression models for more precise threshold prediction |
| PDF Report Export | Download commissioning analysis as formatted PDF |
| Similar Machine Comparison | Compare with existing machines' performance history |
| Multi-Factory Support | Cross-factory data analysis |

---

**JARVIS — Smart Factory Monitoring System v2.0.0**

*New Machine Commissioning Advisor — Implementation Report*

© 2026 Rithish. All rights reserved.

In [ ]:
# Cleanup
db.close()
import os
if os.path.exists('factory_demo.db'):
    os.remove('factory_demo.db')
print('✅ Cleanup complete')